In [ ]:
import optuna
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# загружаем датасет
data = load_iris()
X, y = data.data, data.target

# разбиваем на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# определяем функцию objective для Optuna
def objective(trial):

    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_discrete_uniform('min_samples_split', 0.1, 30)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False]) 

    model = RandomForestClassifier(n_estimators=n_estimators, 
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    bootstrap=bootstrap,
                                    random_state=0)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    accuracy = np.sqrt(accuracy_score(y_test, preds))

    return accuracy # верните значение точности

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# выведите результаты
print('Лучшие параметры:',)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params) 

In [ ]:
import optuna
from optuna.samplers import CmaEsSampler
from optuna.integration.mlflow import MLflowCallback
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import random

# загрузка датасета
digits = load_digits()
X, y = digits.data, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# фиксация seed
# Numpy
np.random.seed(42)
# Python
random.seed(42) 

# настройка Optuna - изменение правила логирования optuna.logging.INFO
optuna.logging.set_verbosity(optuna.logging.INFO) 

storage = "sqlite:///example.db"
study_name = "bagging-optimization-study"

# MLflow Callback
mlflc = MLflowCallback(
    tracking_uri="http://localhost:5000",
    metric_name="accuracy",
)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_samples = trial.suggest_float('max_samples', 0.1, 1.0)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)

    clf = BaggingClassifier(n_estimators=n_estimators, max_samples=max_samples, max_features=max_features, random_state=42)

    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    accuracy = accuracy_score(y_test, preds)

    return accuracy

# создание study с CmaEsSampler и оптимизация
sampler = CmaEsSampler(seed=42)
study = optuna.create_study(direction='maximize', study_name=study_name, storage=storage, sampler=sampler, load_if_exists=True)
study.optimize(objective, n_trials=10, callbacks=[mlflc]) 

# вывод результатов (число итераций и лучшие гиперпараметры)
print('Number of finished trials: ', len(study.trials))
print('Best trial:', study.best_trial.params) 